# Imports:

In [1]:
import numpy as np
import pandas as pd
import math
import datetime
from datetime import datetime

from config import *
from utils import *

# Combining Post April 2019 df and Existing df

In [2]:
existing_df = pd.read_csv(OLD_PATH)
#new_df = pd.read_csv(POST_2019_PATH)
new_df = pd.read_csv(POST_2019_PATH_new)

In [4]:
print('existing df contains '+str(existing_df.Category.nunique())+' unique categories')
print('new (post april 19) df contains '+str(new_df.Category.nunique())+' unique categories')

existing df contains 424 unique categories
new (post april 19) df contains 379 unique categories


In [5]:
final_df = pd.concat([existing_df[existing_df['Date']<'2019-01-15'],new_df])

In [7]:
final_df.tail()

,Date,Category,Category_id,Price,Weight,Indent,Parent,Parent_ID
21418,2023-09-15,New and used motor vehicles,2973.0,127.343,6.987,3.0,Private transportation,3815.0
21419,2023-09-15,Utilities and public transportation,7939.0,252.478,9.322,1.0,All items,8106.0
21420,2023-09-15,Household furnishings and operations,9339.0,148.672,4.282,1.0,All items,8106.0
21421,2023-09-15,Other goods and services,8488.0,543.350,3.207,1.0,All items,8106.0
21422,2023-09-15,Personal care,6986.0,276.813,2.547,2.0,Other goods and services,8488.0


------------------------

# Testing Out The Data:

In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73565 entries, 0 to 21422
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         73565 non-null  object 
 1   Category     73565 non-null  object 
 2   Category_id  73565 non-null  float64
 3   Price        73412 non-null  float64
 4   Weight       73565 non-null  float64
 5   Indent       73565 non-null  float64
 6   Parent       73088 non-null  object 
 7   Parent_ID    73088 non-null  float64
dtypes: float64(5), object(3)
memory usage: 5.1+ MB


In [15]:
# making sure 'all items' is the only category without a parent
final_df[final_df.Parent.isna()].Category.unique()

array(['All items'], dtype=object)

In [10]:
missing_price = list(final_df[final_df.Price.isna()].Category.unique()) # all categories with at least one missing price
missing_prices = final_df[final_df.Category.isin(missing_price)&(final_df.Price.isna())].groupby('Category').agg({'Date':pd.Series.nunique}).reset_index()
missing_prices # df that contains the number of missing price values per category

,Category,Date
0,Admission to sporting events,6
1,Care of invalids and elderly at home,1
2,Distilled spirits away from home,2
3,Domestic services,3
4,Fees for lessons or instructions,1
5,Food at elementary and secondary schools,14
6,Food at employee sites and schools,1
7,Gardening and lawncare services,29
8,Hospital services,1
9,Household operations,12


In [11]:
#removing categories with missing price values
cat_to_remove = list(missing_prices[missing_prices['Date']>0].Category.unique())
final_df = final_df[~final_df['Category'].isin(cat_to_remove)]

In [16]:
print('number of cats in df is: '+str(final_df.Category.nunique()))

number of cats in df is: 418


# Saving Baseline df:

In [17]:
baseline_df = final_df.copy()

In [18]:
baseline_df.to_csv(BASELINE_PATH,index=False)

In [19]:
baseline_df[baseline_df['Category'] == 'Care of invalids and elderly at home'].Date.isna().sum()

0

# Saving HRNN df:

### Fixing Change of Parents/Indents/etc. in Data:

In [20]:
temp = final_df[final_df['Date']>='2023-01-15'].copy()
temp = temp[['Category','Category_id','Indent','Parent','Parent_ID']].drop_duplicates() #using the latest categories, category_ids, indents etc. since they changed over time

In [21]:
agg_temp = temp.groupby('Category_id').agg({'Category': pd.Series.nunique, 'Indent': pd.Series.nunique, 'Parent': pd.Series.nunique, 'Parent_ID': pd.Series.nunique}).reset_index()
agg_temp[(agg_temp['Category'] != 1)|(agg_temp['Indent'] != 1)|(agg_temp['Parent'] != 1 )|(agg_temp['Parent_ID'] != 1)] #checking how many categories have more than one name, indent, paren or parent_id

,Category_id,Category,Indent,Parent,Parent_ID
284,8106.0,1,1,0,0


In [22]:
categories_in_2023 = list(final_df[final_df['Date']>='2023-01-15'].Category_id.unique()) #creating a list of all unique categories in 2023

In [34]:
len(categories_in_2023)

352

In [35]:
fixed_df = final_df[final_df['Category_id'].isin(categories_in_2023)].copy() #using only categories found in 2023

In [36]:
latest_data_per_cat_id = {}
for cat_id in categories_in_2023:
    #saving latest data for updates - list[0] = category name, list[1] = indent, list[2] = parent, list[3] = parent_id
    latest_data_per_cat_id[cat_id] = [temp[temp['Category_id'] == cat_id].Category.values[0],temp[temp['Category_id'] == cat_id].Indent.values[0], temp[temp['Category_id'] == cat_id].Parent.values[0], temp[temp['Category_id'] == cat_id].Parent_ID.values[0]]

In [37]:
for cat_id in categories_in_2023:
    fixed_df.loc[(fixed_df['Category_id']==cat_id), 'Category'] = latest_data_per_cat_id[cat_id][0]
    fixed_df.loc[(fixed_df['Category_id']==cat_id), 'Indent'] = latest_data_per_cat_id[cat_id][1]
    fixed_df.loc[(fixed_df['Category_id']==cat_id), 'Parent'] = latest_data_per_cat_id[cat_id][2]
    fixed_df.loc[(fixed_df['Category_id']==cat_id), 'Parent_ID'] = latest_data_per_cat_id[cat_id][3]

In [38]:
fixed_copy = fixed_df.copy() #creating a copy for tests:

In [39]:
# tests:
for cat_id in fixed_copy.Category_id.unique():
    if cat_id == 8106.0: #all items will be treated different as it doesn't have a parent or parent_id
        continue
    else:
        parent_id = fixed_copy[fixed_copy['Category_id'] == cat_id].Parent_ID.values[0]
        if  fixed_copy[fixed_copy['Category_id'] == parent_id].shape[0] == 0: #if parent isn't in 2022 data then continue- we'll deal later
            continue
        else:
            parent_indent = fixed_copy[fixed_copy['Category_id'] == parent_id].Indent.values[0] 
            fixed_copy.loc[fixed_copy['Category_id'] == cat_id, 'Parent_Indent'] = parent_indent #otherwise, force all categories to have their latest parent to make sure the parent indent is higher (less than) the child's

In [40]:
fixed_copy[fixed_copy['Indent']<=fixed_copy['Parent_Indent']].Category_id.unique() #checking which categories have indent's higher than their parents

array([ 449., 2211.])

In [41]:
print('category 449 has '+str(fixed_copy[fixed_copy['Parent_ID']==449.0].Category_id.nunique())+' distinct children')
print('category 2211 has '+str(fixed_copy[fixed_copy['Parent_ID']==2211.0].Category_id.nunique())+' distinct children')


category 449 has 2 distinct children
category 2211 has 0 distinct children


In [42]:
for cat_id in list(fixed_copy.Category_id.unique()):
    min_date = fixed_copy[fixed_copy['Category_id'] == cat_id].Date.min()
    max_date = fixed_copy[fixed_copy['Category_id'] == cat_id].Date.max()
    
    if min_date > '2019-01-01':
        print(f'category starts after 2019: {cat_id}')
    
    if max_date < '2023-01-01':
        print(f'category starts before 2023: {cat_id}')

In [43]:
#fixed_copy[fixed_copy['Category'] == 'Care of invalids and elderly at home'].Date.unique()

In [44]:
fixed_df_without_indent_issues = fixed_df[~fixed_df['Category_id'].isin([449.0,2211.0])]

In [47]:
fixed_df_without_indent_issues.Category.nunique()

350

In [48]:
fixed_df_without_indent_issues.to_csv(HRNN_PATH,index=False)